In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
import os
import glob
import pandas as pd
import time
from time import sleep
import random

In [ ]:

# Create a DataFrame with account information
accounts_df = pd.DataFrame({
    'username': ['11xerox11@proton.me', '33xerox33@proton.me', '11xerox11@proton.me', '00xerox00@proton.me'],
    'user_agent': ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', 
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', 
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'],
    'proxy': ['proxy1', 'proxy2', 'proxy3', 'proxy4']
})

In [ ]:
def get_verification_code_from_file(download_dir='/Users/adamhunter/Documents/school projs/capstone/data'):
    # Get a list of all the pgp*.txt files in the download directory
    files = glob.glob(os.path.join(download_dir, 'pgp*.txt'))

    # Find the most recent file
    latest_file = max(files, key=os.path.getctime)

    # Open the file and read the contents
    with open(latest_file, 'r') as f:
        contents = f.read()

    # Use regex to find the verification code in the line that starts with "Subject:"
    match = re.search(r'Subject:.*?(\d{6})', contents)
    if match:
        verification_code = match.group(1)
    else:
        print("No verification code found in email.")
        verification_code = None

    return verification_code
def grab_verification(driver, username, password):
    # Save the handle of the original tab
    original_tab = driver.current_window_handle

    # Open a new tab
    driver.execute_script("window.open('');")

    # Switch to the new tab (it's always the last one)
    driver.switch_to.window(driver.window_handles[-1])

    # Navigate to ProtonMail
    driver.get('https://mail.protonmail.com/login')
    # Input username
    username_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'username')))
    sleep(random.random()*2)    
    username_field.send_keys(username)

    password_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'password')))
    sleep(random.random()*3)    
    password_field.send_keys(password)

    # Submit form
    login_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-large')))
    sleep(random.random()*3)    
    login_button.click()
    print('clicked login to email')

    # Click the first email in the inbox
    first_email = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.item-container-wrapper:nth-child(1) .item-subject .max-w100')))
    first_email.click()

    # Click 'More options' button
    more_options_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button:nth-child(11)')))
    more_options_button.click()

    # Click 'View headers' button
    view_headers_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.dropdown-item:nth-child(10) .flex-item-fluid')))
    view_headers_button.click()

    # Click 'Download' button
    download_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.button-solid-norm:nth-child(2)')))
    download_button.click()

    # Switch back to the original tab
    driver.switch_to.window(original_tab)

    sleep(3)

    verification_code = get_verification_code_from_file()

    # Input verification code
    verification_field = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'input__email_verification_pin')))
    sleep(random.random()*3)    
    verification_field.send_keys(verification_code)
    sleep(random.random())

    # Click the verify button
    verify_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, 'email-pin-submit-button')))

    # Create an ActionChains object mouse movement to avoid detection
    actions = ActionChains(driver)

    # Move to the sign in button and click it
    actions.move_to_element(verify_button).click().perform()

In [ ]:
account = accounts_df.iloc[0]
chrome_options = Options()
chrome_options.add_argument(f"user-agent={account['user_agent']}")
#chrome_options.add_argument(f"--proxy-server={account['proxy']}")
prefs = {'download.default_directory' : '/Users/adamhunter/Documents/school projs/capstone/data'}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(service=Service(ChromeDriverManager("117.0.5938.92").install()), options=chrome_options)
driver.maximize_window() #max size for consistency
# Extracting the username and password from the selected account
username = account['username']
password = 'Linked00!'
grab_verification(driver, username, password)